# Vauva.fi ja johdatus aihemallinnukseen

## Data
- Vauva.fi on suosituin suomalaisen aikakauslehden keskustelupalsta
- Aineiston keräys https://github.com/alkukampela/vauvascrape
- 23220 keskusteluketjua, joissa yhteensä 252738 viestiä Aihe vapaa -palstalta

## Kirjastot
- [libvoikko](https://github.com/voikko/corevoikko): Voikko-ohjelmiston avoin kirjasto suomen kielen käsittelyyn
- [gensim](https://github.com/RaRe-Technologies/gensim): "Topic modeling for humans"
- [pyLDAvis](https://github.com/bmabey/pyLDAvis): LDA-algoritmin tulosten visualisointi

In [1]:
import itertools
import pgdb
import libvoikko
from tqdm import tqdm
from gensim.models.phrases import Phrases, Phraser
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning) 

DATABASE = 'vauvafi'
POST_QUERY = 'SELECT id, content FROM posts'
INSERT_NORMALIZED = 'INSERT INTO normalized_posts(id, content) VALUES(%s, %s)'
NORMALIZED_POST_QUERY = 'SELECT * FROM normalized_posts'
NORMALIZED_TOPIC_QUERY = '''
    SELECT topic_id as id, array_concat_agg(normalized_posts.content) AS content
    FROM posts LEFT JOIN normalized_posts
    ON posts.id = normalized_posts.id
    GROUP BY topic_id
    '''

VOIKKO = libvoikko.Voikko('fi')
BATCH_SIZE = 100

with open('stop_words.txt') as f:
    stop_words = [word.rstrip() for word in f]
    
with pgdb.connect(database=DATABASE) as db:
    with db.cursor() as cursor:
        cursor.execute('SELECT COUNT(id) FROM posts')
        POST_COUNT = cursor.fetchone().count
        cursor.execute('SELECT COUNT(id) FROM topics')
        TOPIC_COUNT = cursor.fetchone().count

## Esikäsittely

* ylimääräisten välimerkkien ja sanojen poistaminen
* sanojen perusmuotoistaminen (lemmaus)

In [2]:
def tokenize(text):
    return VOIKKO.tokens(text)

tokenize('Onpa kivaa, kun on perjantai!')

[<Onpa,WORD>,
 < ,WHITESPACE>,
 <kivaa,WORD>,
 <,,PUNCTUATION>,
 < ,WHITESPACE>,
 <kun,WORD>,
 < ,WHITESPACE>,
 <on,WORD>,
 < ,WHITESPACE>,
 <perjantai,WORD>,
 <!,PUNCTUATION>]

In [3]:
def lemmatize(word):
    analysis = VOIKKO.analyze(word)
    try:
        return analysis[0]['BASEFORM'].lower()
    except (IndexError, KeyError):
        return word
    
lemmatize('perjantainakinkohan')

'perjantai'

In [4]:
def is_word(token):
    return token.tokenType == libvoikko.Token.WORD

def is_stop_word(word):
    return word in stop_words or len(word) <= 2

def normalize(text):
    for token in tokenize(text):
        if not is_word(token):
            continue
        word = lemmatize(token.tokenText)
        if word and not is_stop_word(word):
            yield word
            
list(normalize('Onpa kivaa, kun on perjantai!'))

['kiva', 'perjantai']

* dokumenteille käytetään myöhemmin _bag of words_ -esitystä (vektori, jonka kukin komponentti ilmaisee vastaavan sanan esiintymiskertoja dokumentissa), jossa sanojen väliset suhteet menetetään
* halutaan säilyttää merkitykselliset usean sanan yhdistelmät eli ilmaisut, esim. "palkka juoksee" &rightarrow; ilmaisujen mallinnus (_phrase modeling_)
* [gensim](https://radimrehurek.com/gensim/models/phrases.html): yksinkertainen sanojen esiintymiskertoihin perustuva toteutus, jonka syötteenä normalisoidut lauseet:
    
\begin{equation*}
score(w_i, w_j) = \frac{count(w_i, w_j) - \delta}{count(w_i) \times count(w_j)}
\end{equation*}
    

In [5]:
def get_rows(query):
    with pgdb.connect(database=DATABASE) as db:
        with db.cursor() as cursor:
            cursor.execute(query)
            while True:
                rows = cursor.fetchmany(BATCH_SIZE)
                if not rows:
                    break
                yield from rows
            
def get_posts():
    yield from tqdm(get_rows(POST_QUERY), total=POST_COUNT)
    
def split_to_sentences(text):
    return VOIKKO.sentences(text)

def get_normalized_sentences():
    for post in get_posts():
        for sentence in split_to_sentences(post.content):
            yield list(normalize(sentence.sentenceText))
            
for sentence in itertools.islice(get_normalized_sentences(), 49, 52):
    print('- ' + ' '.join(sentence))   

  0%|          | 1/252738 [00:00<21:23:25,  3.28it/s]

- tuntua yökylä ohjelma jakso avautuva paska elämä lapsi hankinta
- lapsi hankkia hoitaa tuntua vastenmielinen
- kirjotetaan tämmösestä tavallinen äiti isä arki


In [6]:
def build_phrase_model(sentences):
    return Phraser(Phrases(sentences))

def pre_process(text, bigram, trigram):
    return list(trigram[bigram[normalize(text)]])

bigram = build_phrase_model(get_normalized_sentences())
trigram = build_phrase_model(bigram[get_normalized_sentences()])

100%|██████████| 252738/252738 [10:25<00:00, 403.74it/s]


In [7]:
' '.join(
    pre_process(
        '''
        Ammuin ilmapistoolilla koulukaveriani jalkaan.
        Sillä oli tiukat farkut jalassa, ei mennyt läpi niistäkään, mutta reiteen tuli mustelma.
        Elettiin kasarin alkua.
        ''',
        bigram,
        trigram
    )
)

'ampua ilmapistooli koulukaveri jalka tiukka_farkku jalka menty reisi mustelma elää kasari alku'

In [8]:
with pgdb.connect(database=DATABASE) as db:
    with db.cursor() as cursor:
        cursor.executemany(INSERT_NORMALIZED, (
            (post.id, pre_process(post.content, bigram, trigram))
            for post in get_posts()
        ))
    db.commit()

100%|██████████| 252738/252738 [13:45<00:00, 306.02it/s]


## Latent Dirichlet Allocation (LDA)
[Blei, David M., Andrew Y. Ng, and Michael I. Jordan. 2003.](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf)

* ohjaamaton (_unsupervised_) algoritmi
* dokumentit (_document_) muodostuvat useasta aiheesta (_topic_), jotka koostuvat useasta sanasta (_word_)

![](img/lda.png)

* generatiivinen prosessi:
    1. dokumentin sanojen lukumäärä $N \sim Poisson(\xi)$
    1. dokumentin aiheiden jakauma $\theta \sim Dirichtlet(\alpha)$
    1. kukin dokumentin N sanasta $w_n$ saadaan valitsemalla:
        1. aihe $z_n \sim Multinomial(\theta)$
        1. sana $w_n \sim p(w_n | z_n, \beta)$ (multinomijakauma, ehdollinen tunnetulle aiheelle $z_n$. $\beta$-matriisi kuvaa kaikkien aiheiden kaikkien sanojen jakautumista)

 &rightarrow; estimoidaan parametrit havaituista dokumenteista

In [9]:
def get_normalized_discussions():
    for discussion in tqdm(get_rows(NORMALIZED_TOPIC_QUERY), total=TOPIC_COUNT):
        yield discussion.content

In [10]:
def build_dictionary():
    dictionary = Dictionary(get_normalized_discussions())
    dictionary.filter_extremes(no_below=20, no_above=0.4)
    dictionary.compactify()
    return dictionary

dictionary = build_dictionary()

100%|█████████▉| 23142/23220 [00:18<00:00, 1243.50it/s]


In [11]:
def get_discussion_bows(dictionary):
    for post in get_normalized_discussions():
        yield dictionary.doc2bow(post)

def build_corpus(dictionary, fname):
    MmCorpus.serialize(fname, get_discussion_bows(dictionary))
    return MmCorpus(fname)

corpus = build_corpus(dictionary, 'vauvafi_corpus.mm')

100%|█████████▉| 23142/23220 [00:22<00:00, 1027.79it/s]


In [12]:
%%time

lda = LdaMulticore(corpus, num_topics=20, id2word=dictionary, workers=3)

CPU times: user 51.7 s, sys: 3.55 s, total: 55.3 s
Wall time: 55.6 s


In [13]:
ldavis_data = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
pyLDAvis.display(ldavis_data)

## Yhteenveto

* esikäsittelyllä suuri merkitys lopputuloksen kannalta
* suomi on vaikeaa, huono suomi vielä vaikeampaa
    * lemmaus/stemmaus, sanojen monimerkityksellisyys
    * valmiita työkaluja/kirjastoja huonommin saatavilla (vastineita Voikolle esim. [nltk](http://www.nltk.org/) ja [spaCy](https://spacy.io/))

* mitä hyötyä tästä oikeastaan oli?
    * visualisointi
    * ymmärrys laajan dokumenttikokonaisuuden aiheista on usein itsessään hyödyllinen (huom. aiheisiin ei kuitenkaan _varsinaisesti_ liity merkitystä)
    * dokumenttien aihejakaumia voidaan hyödyntää bag-of-words-vektoreita pienempiulotteisena esityksinä jonkin muun mallin opetuksessa (esim. luokittelu- ja regressio-ongelmat)
    * aihemallinnusta sovellettu myös esim. geneettiseen dataan, kuviin ja sosiaalisiin verkostoihin ([David M. Blei. 2012.](http://www.cs.columbia.edu/~blei/papers/Blei2012.pdf))